In [1]:
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd

In [2]:
username = 'michals2'
password = 'Nv1VZRmuTxRvJn3u'
host = 'mysql.agh.edu.pl'
port = 3306
database = 'michals2'

connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'

In [3]:
engine = create_engine(connection_string, echo=True)

In [4]:
table_name = 'Perfumes'

df = pd.read_sql_table(table_name, con=engine)
data_list = df.to_dict(orient='records')  # Each row as a dictionary in a list

2025-04-08 19:18:01,175 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-04-08 19:18:01,179 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-08 19:18:01,232 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-04-08 19:18:01,234 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-08 19:18:01,262 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-04-08 19:18:01,263 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-08 19:18:01,312 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-08 19:18:01,314 INFO sqlalchemy.engine.Engine DESCRIBE `michals2`.`Perfumes`
2025-04-08 19:18:01,315 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-08 19:18:01,348 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `michals2`
2025-04-08 19:18:01,349 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-08 19:18:01,375 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `michals2`
2025-04-08 19:18:01,376 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-08 19:18:01,414 INFO sqlalchem

df #use as dataframe </br>
data_list #use as dict

define user prompt

In [10]:
user_prompt = "I like woody, tabacco notes which would be perfect for evening."

content based filtering

In [11]:
df['text_features'] = df['Description'].fillna('') + ' ' + df['Accords'].fillna('') + ' ' + df['Designer'].fillna('') + ' ' + df['TopNotes'].fillna('') + ' ' + df['MiddleNotes'].fillna('') + ' ' + df['BaseNotes'].fillna('')

In [12]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['text_features'])

user_vec = vectorizer.transform([user_prompt])

cos_similarities = cosine_similarity(user_vec, tfidf_matrix).flatten()

In [15]:
top_indices = cos_similarities.argsort()[::-1][:5]
recommended_perfumes = df.iloc[top_indices][['Name', 'Designer', 'Description', 'Accords', 'TopNotes', 'MiddleNotes', 'BaseNotes']]
recommended_perfumes['similarity'] = cos_similarities[top_indices]

In [16]:
print("Top perfume recommendations:")
recommended_perfumes

Top perfume recommendations:


,Name,Designer,Description,Accords,TopNotes,MiddleNotes,BaseNotes,similarity
12,Chilling By The Pacific,The Dua Brand,Chilling By The Pacific by The Dua Brand is a ...,citrus (100%); aromatic (83.5618%); green (59....,"['Citron', 'Coriander', 'Mint', 'Apricot', 'Or...",[],[],0.077640
1098,Dia Woman,Amouage,Dia Woman by Amouage is a Floral fragrance for...,floral (100%); fresh (99.8436%); aldehydic (88...,"['Aldehydes', 'Cyclamen', 'Violet Leaf', 'Sage...","['Peony', 'Turkish Rose Oil', 'Orris Root', 'O...","['White Musk', 'Heliotrope', 'Sandalwood', 'In...",0.076541
52,Sand Desert At Sunset,Zara,Sand Desert At Sunset by Zara is a Oriental Sp...,warm spicy (100%); cinnamon (76.7838%); amber ...,"['Orange', 'Incense']","['Cinnamon', 'Tonka Bean', 'Chocolate']","['Iris', 'Cedar']",0.075917
644,Cinéma,Yves Saint Laurent,Cinéma by Yves Saint Laurent is a Oriental Flo...,floral (100%); vanilla (75.8754%); amber (73.5...,"['Almond Blossom', 'Clementine', 'Cyclamen']","['Jasmine', 'Peony', 'Amaryllis']","['Vanilla', 'Amber', 'Benzoin', 'White Musk']",0.071787
0,Unicorn Milkshake,The Dua Brand,Unicorn Milkshake by The Dua Brand is a Floral...,sweet (100%); vanilla (58.1656%); lactonic (55...,[],[],[],0.071374
